In [1]:
pip install sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

Text Data

In [2]:
text = """
Pinecone is a vector database that lets you search and store embeddings efficiently.
It is often used in semantic search and AI applications like chatbots.
Embeddings are numerical vectors that represent the meaning of text or data.
...
"""


Split Text into Chunks

In [3]:
def split_text(text, max_words=50):
    import re
    sentences = re.split(r'\.|\n', text)
    chunks = []
    chunk = []

    for sentence in sentences:
        words = sentence.strip().split()
        if not words:
            continue
        chunk.extend(words)
        if len(chunk) >= max_words:
            chunks.append(" ".join(chunk))
            chunk = []

    if chunk:
        chunks.append(" ".join(chunk))

    return chunks

chunks = split_text(text)
print(f"Total Chunks: {len(chunks)}")


Total Chunks: 1


Embed Text Using Local Model

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Store Vectors in FAISS

In [5]:
import faiss
import numpy as np

dimension = embeddings.shape[1]  # e.g., 384
index = faiss.IndexFlatL2(dimension)  # L2 = Euclidean similarity
index.add(np.array(embeddings))  # store the vectors


Create a Search Function

In [6]:
def search(query, k=3):
    query_vec = model.encode([query])
    D, I = index.search(query_vec, k)
    return [chunks[i] for i in I[0]]


Build a Simple Q&A Loop

In [ ]:
while True:
    user_input = input("\nAsk a question (or type 'exit'): ")
    if user_input.lower() == "exit":
        break

    results = search(user_input)
    print("\n📚 Top relevant texts:")
    for i, res in enumerate(results):
        print(f"{i+1}. {res}")



Ask a question (or type 'exit'): which data base

📚 Top relevant texts:
1. Pinecone is a vector database that lets you search and store embeddings efficiently It is often used in semantic search and AI applications like chatbots Embeddings are numerical vectors that represent the meaning of text or data
2. Pinecone is a vector database that lets you search and store embeddings efficiently It is often used in semantic search and AI applications like chatbots Embeddings are numerical vectors that represent the meaning of text or data
3. Pinecone is a vector database that lets you search and store embeddings efficiently It is often used in semantic search and AI applications like chatbots Embeddings are numerical vectors that represent the meaning of text or data

Ask a question (or type 'exit'): pinecone is what

📚 Top relevant texts:
1. Pinecone is a vector database that lets you search and store embeddings efficiently It is often used in semantic search and AI applications like chatbo

chatbot now:
Stores and searches our own data
Finds the most relevant passages
Can be improved with a simple LLM later